In [173]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from google.cloud import bigquery
from google.oauth2 import service_account
import requests
from sklearn.preprocessing import StandardScaler


In [189]:
PROJECT_ID = 'lewagonbootcamp-371116'
credentials = service_account.Credentials.from_service_account_file('authenticate-gcs.json')


In [552]:
bq_client = bigquery.Client(credentials= credentials,project=PROJECT_ID)

In [553]:
pd.set_option('display.max_columns', None)

In [554]:
result = bq_client.query(f"""
    SELECT *
    FROM moda_data.moda_asos_data
    WHERE display_image IS NOT NULL
    """)
df = result.to_dataframe()

In [555]:
df.drop_duplicates(inplace=True, subset='id')

In [556]:
df.shape

(799, 33)

In [557]:
df_product_section = 'upper'
if df_product_section=='full body':
    df_product_section = 'full'
df_product_cat = 'sweaters'


In [558]:
df_product = df[df[f'{df_product_section}_category']==df_product_cat]
if df_product_section != 'full':
    df_product_knn = df_product.iloc[:,np.r_[0,13,14,15,16,17,19,20,21,22,31]]
else:
    df_product_knn = df_product.iloc[:,np.r_[0,24,25,26,27,28,29,30]]

In [559]:
color_upper = df_product_knn.loc[:,'upper_color'].apply(lambda x : x.strip("[]").split()).values

color_lower = df_product_knn.loc[:,'lower_color'].apply(lambda x : x.strip("[]").split()).values


In [560]:
color_df = pd.DataFrame({"upper_r": color_upper[i][0], "upper_b": color_upper[i][1], "upper_g": color_lower[i][2], "lower_r": color_lower[i][0], "lower_b": color_lower[i][1], "lower_g": color_lower[i][2]} for i in range(len(color_upper)))

scaler = StandardScaler()
# Fit the scaler to the data
color_df = pd.DataFrame(scaler.fit_transform(color_df))
color_df.columns = scaler.get_feature_names_out()


In [561]:
color_df.tail()

,upper_r,upper_b,upper_g,lower_r,lower_b,lower_g
46,0.988522,1.028947,0.908146,0.839700,0.882379,0.908146
47,1.021034,1.039885,1.033016,0.975329,1.015486,1.033016
48,0.966848,0.963325,0.771924,0.839700,0.826918,0.771924
49,0.630895,0.668022,0.204333,0.071139,0.128105,0.204333
50,0.923499,0.941450,1.044368,0.975329,1.015486,1.044368


In [562]:
df_product_knn.drop(columns=['upper_color','lower_color'],inplace=True)

/var/folders/xy/67_kk4593zb8wd2k_6hccr880000gn/T/ipykernel_63086/107255524.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_product_knn.drop(columns=['upper_color','lower_color'],inplace=True)


In [563]:
df_product_knn

,id,upper_design,upper_sleeves,upper_neckline,upper_fabric,upper_fit,lower_design,lower_fabric,lower_fit
11,200325521,solid,sleeveless,crew_neckline,cotton,conventional,solid,cotton,conventional
41,13268987,solid,short_sleeve,v_neckline,cotton,conventional,solid,cotton,conventional
60,200923372,graphic,long_sleeve,crew_neckline,cotton,conventional,solid,denim,conventional
95,200860562,striped,long_sleeve,crew_neckline,knit,conventional,solid,denim,conventional
122,201266553,solid,long_sleeve,v_neckline,denim,conventional,graphic,denim,conventional
138,201248864,solid,long_sleeve,crew_neckline,knit,tight,solid,knit,tight
141,12942320,solid,short_sleeve,crew_neckline,denim,conventional,solid,denim,tight
153,200817420,solid,long_sleeve,crew_neckline,cotton,tight,solid,leather,conventional
191,201306041,floral,short_sleeve,crew_neckline,chiffon,conventional,floral,denim,conventional
220,20026914,solid,long_sleeve,crew_neckline,denim,tight,solid,cotton,conventional


In [564]:
encoder = OneHotEncoder(sparse=False)
encoder.fit(df_product_knn.iloc[:,1:])
df_product_knn_encoded = pd.DataFrame(encoder.transform(df_product_knn.iloc[:,1:]))
df_product_knn_encoded.columns = encoder.get_feature_names_out()


In [565]:
df_product_knn_encoded

,upper_design_embroidered,upper_design_floral,upper_design_graphic,upper_design_pleated,upper_design_solid,upper_design_striped,upper_sleeves_long_sleeve,upper_sleeves_short_sleeve,upper_sleeves_sleeveless,upper_neckline_crew_neckline,upper_neckline_square_neckline,upper_neckline_v_neckline,upper_fabric_chiffon,upper_fabric_cotton,upper_fabric_denim,upper_fabric_faux,upper_fabric_knit,upper_fabric_leather,upper_fit_conventional,upper_fit_loose,upper_fit_tight,lower_design_embroidered,lower_design_floral,lower_design_graphic,lower_design_pleated,lower_design_solid,lower_fabric_chiffon,lower_fabric_cotton,lower_fabric_denim,lower_fabric_faux,lower_fabric_knit,lower_fabric_leather,lower_fit_conventional,lower_fit_tight
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
5,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
7,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
8,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
9,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [566]:
df_product_knn_encoded=df_product_knn_encoded.merge(color_df, left_index=True, right_index=True)

In [567]:
df_product_knn_encoded

,upper_design_embroidered,upper_design_floral,upper_design_graphic,upper_design_pleated,upper_design_solid,upper_design_striped,upper_sleeves_long_sleeve,upper_sleeves_short_sleeve,upper_sleeves_sleeveless,upper_neckline_crew_neckline,upper_neckline_square_neckline,upper_neckline_v_neckline,upper_fabric_chiffon,upper_fabric_cotton,upper_fabric_denim,upper_fabric_faux,upper_fabric_knit,upper_fabric_leather,upper_fit_conventional,upper_fit_loose,upper_fit_tight,lower_design_embroidered,lower_design_floral,lower_design_graphic,lower_design_pleated,lower_design_solid,lower_fabric_chiffon,lower_fabric_cotton,lower_fabric_denim,lower_fabric_faux,lower_fabric_knit,lower_fabric_leather,lower_fit_conventional,lower_fit_tight,upper_r,upper_b,upper_g,lower_r,lower_b,lower_g
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.923499,0.930513,1.033016,0.964027,1.004394,1.033016
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.620058,0.657085,0.442721,0.285884,0.316673,0.442721
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,-0.810452,-0.764741,1.067072,0.997934,1.037671,1.067072
3,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.825964,0.777394,-1.225998,-1.296450,-1.225152,-1.225998
4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.901825,0.941450,-1.430330,-1.545102,-1.424813,-1.430330
5,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,-1.482358,-1.420968,-1.384923,-1.533800,-1.413721,-1.384923
6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.782615,0.821142,0.227037,-0.335747,-0.071556,0.227037
7,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,-1.135568,-1.125666,-1.350868,-1.409473,-1.313890,-1.350868
8,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.468337,0.711771,1.033016,0.376303,0.627257,1.033016
9,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.322778,-0.283507,0.794628,0.749281,0.793641,0.794628


In [568]:
df_product_knn_encoded = df_product_knn_encoded.loc[:, df_product_knn_encoded.columns.str.contains(df_product_section)]

In [569]:
df_product_knn_encoded

,upper_design_embroidered,upper_design_floral,upper_design_graphic,upper_design_pleated,upper_design_solid,upper_design_striped,upper_sleeves_long_sleeve,upper_sleeves_short_sleeve,upper_sleeves_sleeveless,upper_neckline_crew_neckline,upper_neckline_square_neckline,upper_neckline_v_neckline,upper_fabric_chiffon,upper_fabric_cotton,upper_fabric_denim,upper_fabric_faux,upper_fabric_knit,upper_fabric_leather,upper_fit_conventional,upper_fit_loose,upper_fit_tight,upper_r,upper_b,upper_g
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.923499,0.930513,1.033016
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.620058,0.657085,0.442721
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.810452,-0.764741,1.067072
3,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.825964,0.777394,-1.225998
4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.901825,0.941450,-1.430330
5,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,-1.482358,-1.420968,-1.384923
6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.782615,0.821142,0.227037
7,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.135568,-1.125666,-1.350868
8,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.468337,0.711771,1.033016
9,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.322778,-0.283507,0.794628


In [570]:
model = KNeighborsClassifier(n_neighbors=10)

In [571]:
X =df_product_knn_encoded

In [572]:
y = np.ones((X.shape[0]))

In [573]:
model.fit(X, y)

KNeighborsClassifier(n_neighbors=10)

In [574]:
X.loc[10]

upper_design_embroidered          0.000000
upper_design_floral               0.000000
upper_design_graphic              0.000000
upper_design_pleated              0.000000
upper_design_solid                1.000000
upper_design_striped              0.000000
upper_sleeves_long_sleeve         1.000000
upper_sleeves_short_sleeve        0.000000
upper_sleeves_sleeveless          0.000000
upper_neckline_crew_neckline      1.000000
upper_neckline_square_neckline    0.000000
upper_neckline_v_neckline         0.000000
upper_fabric_chiffon              0.000000
upper_fabric_cotton               1.000000
upper_fabric_denim                0.000000
upper_fabric_faux                 0.000000
upper_fabric_knit                 0.000000
upper_fabric_leather              0.000000
upper_fit_conventional            1.000000
upper_fit_loose                   0.000000
upper_fit_tight                   0.000000
upper_r                           0.890987
upper_b                           0.810205
upper_g    

In [575]:
nearest = model.kneighbors(X=pd.DataFrame(X.loc[1]).T, n_neighbors=3, return_distance=False)[0]

In [576]:
print(nearest)

[ 1 49 15]


In [577]:
df_product_knn.iloc[1]

id                    13268987
upper_design             solid
upper_sleeves     short_sleeve
upper_neckline      v_neckline
upper_fabric            cotton
upper_fit         conventional
lower_design             solid
lower_fabric            cotton
lower_fit         conventional
Name: 41, dtype: object

In [578]:
df_product_knn.iloc[nearest]

,id,upper_design,upper_sleeves,upper_neckline,upper_fabric,upper_fit,lower_design,lower_fabric,lower_fit
41,13268987,solid,short_sleeve,v_neckline,cotton,conventional,solid,cotton,conventional
1208,201259155,solid,long_sleeve,v_neckline,cotton,conventional,solid,cotton,conventional
365,10898747,solid,short_sleeve,v_neckline,knit,conventional,solid,chiffon,conventional


In [31]:
data = pd.read_csv('Data/data_full.csv')

In [3]:
data.head()

,Unnamed: 0,img,category,section,floral,graphic,striped,embroidered,pleated,solid,...,y5,v6,x6,y6,v7,x7,y7,v8,x8,y8
0,0,Sweet_Crochet_Blouse/img_00000070.jpg,Blouse,upper,0,0,0,1,0,0,...,273.0,0.0,212.0,267.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,Classic_Pencil_Skirt/img_00000010.jpg,Skirt,lower,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,Strapless_Diamond_Print_Dress/img_00000038.jpg,Dress,full body,0,1,0,0,0,0,...,99.0,0.0,138.0,102.0,0.0,89.0,276.0,0.0,159.0,277.0
3,3,Mid-Rise_-_Acid_Wash_Skinny_Jeans/img_00000010...,Jeans,lower,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,Zippered_Single-Button_Blazer/img_00000078.jpg,Blazer,upper,0,0,0,0,0,1,...,281.0,0.0,186.0,280.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
encoder = OneHotEncoder(sparse=False)

In [40]:
encoder.fit(data[['section']])

OneHotEncoder(sparse=False)

In [43]:
section_hot = pd.DataFrame(encoder.transform(data[['section']]))

In [44]:
section_hot.columns = encoder.get_feature_names()

/home/nigel/.pyenv/versions/3.10.6/envs/taxifare-env/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [37]:
encoder = OneHotEncoder(sparse=False)
encoder.fit(data[['category']])
category_hot = pd.DataFrame(encoder.transform(data[['category']]))
category_hot.columns = encoder.get_feature_names_out()


In [46]:
data = data.drop(columns='x_1 y_1 x_2 y_2 v1 x1 y1 v2 x2 y2 v3 x3 y3 v4 x4 y4 v5 x5 y5 v6 x6 y6 v7 x7 y7 v8 x8 y8'.split())

In [47]:
data = data.drop(columns=['Unnamed: 0', 'category', 'section'])

In [52]:
data_full = pd.merge(left=data, right=cats, left_index=True, right_index=True)

In [53]:
data_full = pd.merge(left=data_full, right=category_hot)

,img,floral,graphic,striped,embroidered,pleated,solid,lattice,long_sleeve,short_sleeve,...,category_Shorts,category_Skirt,category_Sweater,category_Sweatpants,category_Sweatshorts,category_Tank,category_Tee,category_Top,category_Trunks,category_Turtleneck
0,Sweet_Crochet_Blouse/img_00000070.jpg,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Classic_Pencil_Skirt/img_00000010.jpg,0,0,0,0,0,1,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Strapless_Diamond_Print_Dress/img_00000038.jpg,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mid-Rise_-_Acid_Wash_Skinny_Jeans/img_00000010...,0,0,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Zippered_Single-Button_Blazer/img_00000078.jpg,0,0,0,0,0,1,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,Embroidered_Peasant_Tank/img_00000044.jpg,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
19996,Tasmanian_Devil_Sweater/img_00000048.jpg,0,0,0,0,0,1,0,1,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19997,Lace-Sleeved_Raglan_Henley/img_00000022.jpg,0,0,1,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19998,Floral_Mosaic_Print_Jumpsuit/img_00000017.jpg,1,0,0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
data_full.columns

Index(['img', 'floral', 'graphic', 'striped', 'embroidered', 'pleated',
       'solid', 'lattice', 'long_sleeve', 'short_sleeve', 'sleeveless',
       'maxi_length', 'mini_length', 'no_dress', 'crew_neckline', 'v_neckline',
       'square_neckline', 'no_neckline', 'denim', 'chiffon', 'cotton',
       'leather', 'faux', 'knit', 'tight', 'loose', 'conventional',
       'category_Anorak', 'category_Blazer', 'category_Blouse',
       'category_Bomber', 'category_Button-Down', 'category_Caftan',
       'category_Capris', 'category_Cardigan', 'category_Chinos',
       'category_Coat', 'category_Coverup', 'category_Culottes',
       'category_Cutoffs', 'category_Dress', 'category_Flannel',
       'category_Halter', 'category_Henley', 'category_Hoodie',
       'category_Jacket', 'category_Jeans', 'category_Jeggings',
       'category_Jersey', 'category_Jodhpurs', 'category_Joggers',
       'category_Jumpsuit', 'category_Kaftan', 'category_Kimono',
       'category_Leggings', 'category_Onesie',

In [56]:
model = KNeighborsClassifier(n_neighbors=10)

In [57]:
X = data_full.drop(columns='img')

In [58]:
y = data_full['img']

In [59]:
model.fit(X, y)

KNeighborsClassifier(n_neighbors=10)

In [71]:
nearest = model.kneighbors(X=pd.DataFrame(X.loc[10]).T, n_neighbors=5, return_distance=False)[0]

In [72]:
len(nearest)

5